# Preprocesamiento de datos

Para evitar tener que preprocesar los datos cada vez que abrimos el Google Colab vamos a realizar tanto particionado de los datos como su preprocesamiento en este notebook aparte.

## Fase 1 - Cargar el dataset

Lo primero es cargar el dataset con Pandas y seleccionar la cantidad de filas que queremos usar:

In [5]:
import pandas as pd

raw_data = pd.read_csv("dataset_definitivo.csv")
print("general counts")
print(raw_data['personality'].value_counts())
training_data = raw_data[:10000]

print("used dataset info")
print(training_data.shape)
print(training_data.head())
print(training_data['personality'].value_counts())

general counts
personality
7     257836
3     218768
8     188915
4     176474
11    133049
12    121199
15     78908
5      61032
6      59402
2      55199
16     50715
1      44130
10     25463
9      18853
14     15633
13     13672
Name: count, dtype: int64
used dataset info
(10000, 2)
   personality                                               post
0            4  @Pericles216 @HierBeforeTheAC @Sachinettiyil T...
1            4  @HierBeforeTheAC @Pericles216 @Sachinettiyil A...
2            4  @HierBeforeTheAC @Pericles216 @Sachinettiyil Y...
3            4  @HierBeforeTheAC @Pericles216 @Sachinettiyil Y...
4            4  @HierBeforeTheAC @Pericles216 @Sachinettiyil T...
personality
4     4765
8     4064
16    1171
Name: count, dtype: int64


## Fase 2 - Filtrado de Stopwords, Tokenización y Stemming

El preprocesamiento de texto se realiza usando las stopwords y el tokenizer de la librería de lenguaje natural de Python. Primero definimos las stopwords, que serán las de la librería de Python y las personalidades Myers-Briggs para evitar introducir sesgos, como que, si se menciona una personalidad, que esta no se tenga en cuenta para decidir la del autor. Luego se usa el PorterStemmer para obtener la “raíz” de las palabras. Entonces el resultado es una nueva columna que contiene los posts preprocesados.

In [6]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from tqdm.contrib.concurrent import process_map
from multiprocessing import cpu_count
import re

import nltk
nltk.download('punkt')
nltk.download('stopwords')

ps = PorterStemmer()


STOPS = set(stopwords.words("english"))
CUSTOM_STOPS = set(['istj', 'isfj', 'infj', 'intj',
                    'istp', 'isfp', 'infp', 'intp',
                    'estp', 'esfp', 'enfp', 'entp',
                    'estj', 'esfj', 'enfj', 'entj'])

def process_text(post):
    try:
        post = post.lower()
        post = re.sub('https?://[^\s<>"]+|www\.[^\s<>"]+',' ',post)
        post = re.sub('[^0-9a-z]',' ',post)
        return " ".join(
            [ps.stem(w) for w in word_tokenize(post)
             if not w in STOPS and w not in CUSTOM_STOPS])
    except:
        print("problem with: ", post)

preprocessedData = training_data.loc[:]

num_processes = cpu_count()

preprocessedData['processed_text'] = process_map(
                    process_text, training_data['post'],
                    max_workers=num_processes, chunksize=10)

preprocessedData

[nltk_data] Downloading package punkt to /home/mario/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/mario/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


  0%|          | 0/10000 [00:00<?, ?it/s]

,personality,post,processed_text
0,4,@Pericles216 @HierBeforeTheAC @Sachinettiyil T...,pericles216 hierbeforetheac sachinettiyil pope...
1,4,@HierBeforeTheAC @Pericles216 @Sachinettiyil A...,hierbeforetheac pericles216 sachinettiyil perp...
2,4,@HierBeforeTheAC @Pericles216 @Sachinettiyil Y...,hierbeforetheac pericles216 sachinettiyil open...
3,4,@HierBeforeTheAC @Pericles216 @Sachinettiyil Y...,hierbeforetheac pericles216 sachinettiyil know...
4,4,@HierBeforeTheAC @Pericles216 @Sachinettiyil T...,hierbeforetheac pericles216 sachinettiyil like...
...,...,...,...
9995,16,@threadreaderapp unroll,threadreaderapp unrol
9996,16,In my personal life being an #empath #HSP mean...,person life empath hsp mean care friend famili...
9997,16,Being a #HSP #empath means in my professional ...,hsp empath mean profession life person capabl ...
9998,16,But more and more over the past years I have c...,past year chang longer afraid tell peopl sensit


## Fase 3 - Particionar el dataset para entrenamiento y validación
Se divide el dataset en 2 para tener 80% datos de entrenamiento y 20% de prueba

In [10]:
from sklearn.model_selection import train_test_split

X = preprocessedData['processed_text']
Y = preprocessedData['personality']

X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42)

## Fase 4 - Bolsa de palabras

Se crea una bolsa de palabras utilizando TfidVectorizer, que se basa en la frecuencia de las palabras para determinar su importancia.

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Datos de entrenamiento
bagOfWordsModel = TfidfVectorizer()
X_train = bagOfWordsModel.fit_transform(X_train)
print("X_train bag of words:")
print(X_train.shape)

# Datos pruebas
X_test = bagOfWordsModel.transform(X_test)
print("X_test bag of words:")
print(X_test.shape)

X_train bag of words:
(8000, 12043)
X_test bag of words:
(2000, 12043)


Utilizamos TruncatedSVD para reducir la dimensionalidad de los datos de entrenamiento y prueba, permitiendo así trabajar con un número menor de características manteniendo la información relevante para el modelo. Esto puede ser útil en casos donde se tiene un alto número de características y se desea reducir la complejidad del modelo o mejorar los tiempos de entrenamiento.

In [ ]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD()
X_train = svd.fit_transform(X_train)
X_test = svd.transform(X_test)

print(X_train.shape)
print(X_test.shape)

## Fase 5 - Guardar el resultado

Utilizamos joblib para guardar estas bolsas de palabras para poder usarlas para entrenamiento sin tener que repetir el proceso.

In [12]:
from tqdm.contrib.concurrent import process_map,thread_map
from joblib import dump

def multiargs_dump(args_tuple):
    dump(args_tuple[0], args_tuple[1])


thread_map(multiargs_dump,
    (
        (X_train, "X_train.lzma"),
        (X_test, "X_test.lzma"),
        (y_train, "y_train.lzma"),
        (y_test, "y_test.lzma"),
        (bagOfWordsModel, "bag_of_words.lzma")
    ))


  0%|          | 0/5 [00:00<?, ?it/s]

[None, None, None, None, None]